<a href="https://colab.research.google.com/github/enoch20142009/stock-market/blob/main/notebooks/financial_news_stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

# Import Packages

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import requests
from io import StringIO
import json

#Fin Data Sources
import yfinance as yf
import time
from datetime import date

# Ingest Datasets

In [ ]:
# Extract AAPL, MSFT, TSLA price information from yfinance
tickers = ['AAPL', 'MSFT', 'TSLA']
stock_df = yf.download(tickers, start="2023-01-01", end="2023-12-31")
stock_df = stock_df.stack(level=1).reset_index().rename(columns={"level_1": "ticker"})

# Log the stock DataFrame
log_audit('stock_df', 'Read in stock_df from yfinance package', stock_df)

#Save the DataFrame to a CSV file for reproducibility
stock_df.to_csv("stock_df.csv", index=False)

/tmp/ipython-input-4-3503983122.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_df = yf.download(tickers, start="2023-01-01", end="2023-06-01")
[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-4-3503983122.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stock_df = stock_df.stack(level=1).reset_index().rename(columns={"level_1": "ticker"})


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2023-01-03,AAPL,123.470612,129.226052,122.582119,128.613985,112117500
1,2023-01-03,MSFT,234.808975,240.856103,232.672380,238.239276,25740000
2,2023-01-03,TSLA,108.099998,118.800003,104.639999,118.470001,231402800
3,2023-01-04,AAPL,124.744118,127.014709,123.480487,125.267339,89113600
4,2023-01-04,MSFT,224.537674,228.232587,221.460205,227.654340,50623400


In [ ]:
# Read in financial news dataset
from pandas import json_normalize

with open("polygon_news_sample.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Convert nested JSON news dataset into structured tabular format
exploded_df = df.explode('insights').reset_index(drop=True)
exploded_df = json_normalize(exploded_df['insights'])
news_df = pd.concat([df.drop(columns=['insights']), exploded_df], axis=1)

# Only keep useful columns
news_df = news_df[['published_utc', 'description', 'title', 'ticker', 'sentiment', 'sentiment_reasoning']]
news_df.isnull().any()

# Filter to only 'AAPL', 'MSFT', 'TSLA'
tickers = ['AAPL', 'MSFT', 'TSLA']
filter = news_df['ticker'].isin(tickers)
news_df = news_df[filter]

# Log the stock DataFrame
log_audit('news_df', 'Read in news_df from raw JSON file', news_df)

,published_utc,description,title,ticker,sentiment,sentiment_reasoning
109,2023-10-31T15:39:00Z,Arch Capital Group Ltd. reported strong third-...,Arch Capital (ACGL) Q3 Earnings Top on Solid U...,TSLA,positive,Tesla's stock has rebounded 99% year-to-date a...
110,2023-06-22T13:00:13Z,Cisco Systems (CSCO) has been a trending stock...,"Here is What to Know Beyond Why Cisco Systems,...",TSLA,positive,The article expects Tesla's Investor Day event...
144,2023-11-22T14:38:00Z,The trucking industry is benefiting from impro...,The Zacks Analyst Blog Highlights Knight-Swift...,TSLA,positive,Tesla is the largest producer of electric vehi...
145,2023-12-22T11:00:00Z,The article discusses the latest research repo...,"The Zacks Analyst Blog Highlights Alphabet, Co...",MSFT,positive,Microsoft is included in the 'Magnificent 7' t...
168,2023-02-24T14:55:09Z,Pegasystems (PEGA) has formed a hammer chart p...,"Pegasystems (PEGA) Could Find a Support Soon, ...",AAPL,positive,Apple is praised for its effective stock repur...


# Data Hygiene work

In [ ]:
# Ensure no missing values/invalid values and set up audit trail to track for deleted rows, remove neutral sentiment
news_df = news_df.dropna()
log_audit('news_df', 'Remove all NaN value and invalid values (if any)', news_df)

# Ensure no invalid values (Validation check)
stock_df = stock_df.dropna()
filter = (stock_df['Close'] > 0) & (stock_df['High'] > 0) & (stock_df['Low'] > 0) & (stock_df['Open'] > 0)& (stock_df['Volume'] > 0)
stock_df = stock_df[filter]
log_audit('stock_df', 'Remove all NaN value and invalid values (if any)', stock_df)

In [ ]:
# Ensure the consistency of date format in stock_df
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Convert date format on news_df_final
news_df['published_utc'] = pd.to_datetime(news_df['published_utc'])
news_df['Date'] = news_df['published_utc'].dt.date
news_df.drop('published_utc', axis=1, inplace=True)
news_df['Date'] = pd.to_datetime(news_df['Date'])
log_audit('news_df', 'Add Date column from published_utc to match the date format of stock_df', news_df)

/tmp/ipython-input-54-3707434721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['published_utc'] = pd.to_datetime(news_df['published_utc'])
/tmp/ipython-input-54-3707434721.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['Date'] = news_df['published_utc'].dt.date
/tmp/ipython-input-54-3707434721.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [ ]:
# Drop duplicate just in case of multiple entries and only keep the first news for the day for simplicity
stock_df = stock_df.drop_duplicates(subset=['Date', 'Ticker'])
log_audit('stock_df', 'Remove duplicate rows (if any)', stock_df)

news_df = news_df.drop_duplicates(subset=['Date', 'ticker'])
log_audit('news_df', 'Remove duplicate rows (if any) to just keep one news for each day for simplicity', news_df)

# Dataset Integration

In [ ]:
# Merge the two dataset
merged_df = pd.merge(news_df, stock_df, how='inner', left_on=['ticker', 'Date'], right_on=['Ticker', 'Date'])

# Add daily difference column
merged_df['daily_diff'] = merged_df['Close'] - merged_df['Open']
merged_df.drop('ticker', axis=1, inplace=True)
log_audit('merged_df', 'Create merged_df by merging stock_df and news_df, also create a column daily_diff to capture daily price movement', merged_df)

# Export the merged dataset into csv
merged_df.to_csv("merged_dataset.csv", index=False)

,description,title,sentiment,sentiment_reasoning,Date,Ticker,Close,High,Low,Open,Volume
0,Pegasystems (PEGA) has formed a hammer chart p...,"Pegasystems (PEGA) Could Find a Support Soon, ...",positive,Apple is praised for its effective stock repur...,2023-02-24,AAPL,145.055038,145.529620,144.076201,145.450520,55469600
1,Orange is holding its annual general meeting o...,Orange: Assemblée générale du 23 mai 2023 - Mo...,positive,Berkshire invested in Apple when it was tradin...,2023-04-21,AAPL,163.158478,164.572339,162.634458,163.188138,58337300
2,Analyst downgrades SiTime due to lack of visib...,SiTime Downgraded: Lack Of Visibility Of Oscil...,neutral,The article does not provide a specific sentim...,2023-05-04,MSFT,300.077789,302.386766,298.102873,300.893285,22519900
3,ProMIS Neurosciences presented preclinical dat...,ProMIS Neurosciences Presents Preclinical Data...,positive,Microsoft is described as a legend of Wall Str...,2023-04-24,MSFT,276.850494,279.974997,273.853757,277.164915,26611000
4,"U.S. Bancorp is a strong dividend play, with a...",U.S. Bancorp (USB) is a Top Dividend Stock Rig...,neutral,Tesla is mentioned as a better-established bus...,2023-01-26,TSLA,160.270004,161.419998,154.759995,159.970001,234815100
5,"Hologic, a medical device maker, has seen its ...","Hologic, Inc. (HOLX) Is a Trending Stock: Fact...",neutral,Microsoft's GitHub was mentioned as a competit...,2023-02-17,MSFT,253.554413,255.548969,251.530382,254.861209,30000100
6,Osisko Development Corp. is seeking approval t...,Osisko Development Announces Proposed Reductio...,positive,Apple is the world's third-largest video game ...,2023-03-14,AAPL,150.868698,151.669558,148.406796,149.573478,73695900
7,"Insiders have been selling shares in Crocs, Dr...","Crocs, Lamb Weston And These 3 Stocks Insiders...",neutral,The article mentions Microsoft as getting atte...,2023-01-06,MSFT,220.450668,221.264141,214.981803,218.559110,43613600
8,The article discusses how Amazon and Nvidia co...,These 2 High-Growth Stocks Could Power the Bul...,positive,Tesla's stock was up around 2.7% during the da...,2023-03-20,TSLA,183.250000,186.440002,176.350006,178.080002,129684400
9,Crocs and Dollar General are two stocks that c...,2 Growth Stocks That Could Soar in 2023,positive,Datadog is extending its collaboration with Mi...,2023-01-11,MSFT,231.074829,231.251237,226.507626,226.684034,28669300
